In [1]:
import scipy.sparse as sp
import numpy as np
import pandas as pd
import random
import json
import re

In [2]:
featureMatrix = sp.load_npz("feature_matrix.npz")

In [3]:
featureMatrix

<19579x24951 sparse matrix of type '<class 'numpy.int64'>'
	with 250731 stored elements in Compressed Sparse Row format>

In [4]:
# Testing if generated matrix was okay or not
np.where(featureMatrix.toarray()[0] == 1)

(array([  453,  1286,  1636,  3702,  6102,  6862,  8189, 10730, 13337,
        13640, 13875, 16003, 16564, 17069, 18513, 19478, 19625, 23225,
        24168, 24416, 24641]),)

#### The code above was a test to check if everything seems fine after loading the featureMatrix from disk. Looks good to me

### TRAINING: 

In [5]:
train_df = pd.read_csv("dataset/train.csv")

In [6]:
num_examples, num_vocab_words = featureMatrix.shape

In [7]:
class_labels = list(train_df["author"].unique())
class_labels

['EAP', 'HPL', 'MWS']

In [8]:
labelMap = {}
for idx, label in enumerate(class_labels):
    labelMap[label] = idx
labelMap

{'EAP': 0, 'HPL': 1, 'MWS': 2}

In [9]:
all_indices = [i for i in range(num_examples)]

In [10]:
random.shuffle(all_indices)
num_training_examples = int(num_examples * 0.7)
num_test_examples = num_examples - num_training_examples
train_indices, test_indices = all_indices[:num_training_examples], all_indices[num_training_examples:]

In [11]:
with open("word_index.json", "r") as file:
    wordIndex = json.load(file)

In [46]:
class NaiveBayesClassifier():
    def __init__(
        self, 
        num_training_examples, 
        num_test_examples, 
        train_indices,
        test_indices, 
        train_df, 
        featureMatrix
    ):
        self.num_train_examples = num_training_examples
        self.num_test_examples = num_test_examples
        self.train_indices = train_indices
        self.test_indices = test_indices
        self.dataframe = train_df
        self.featureMatrix = featureMatrix

    def train(self, ):
        self.trainFeatureMatrix = sp.csr_matrix(np.array(self.featureMatrix.toarray()[train_indices]))
        

    def evaluateExample(self):
        """
        Parameters
        ------------
        featureVector : (X1,X2,X3,.....Xn) [n = num_vocab_words]


        Returns
        ------------
        The predicted label_id of the example
        
        """
        pass

    def test(self, ):
        """
        Predict Accuracy of All test samples
        """
        pass

In [47]:
NBClassifier = NaiveBayesClassifier(
    num_training_examples, 
    num_test_examples, 
    train_indices, 
    test_indices, 
    train_df, 
    featureMatrix
)

In [48]:
NBClassifier.train()

In [52]:
NBClassifier.trainFeatureMatrix

<13705x24951 sparse matrix of type '<class 'numpy.int64'>'
	with 175127 stored elements in Compressed Sparse Row format>